## Dataset & Dataloader

Датасет хранит все данные, а даталоудер может по ним итерироваться, управлять созданием батчей, трансформировать данные и т.д. 

In [ ]:
!pip install torchmetrics

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F 
import torch.optim as optim
from torchmetrics import Accuracy
from sklearn.model_selection import train_test_split

Pandas для нас необязателен, но его удобно использовать. 

У нас есть датасет из прошлого семестра про качество вина. Подгрузим его. 

In [ ]:
!wget https://raw.githubusercontent.com/rsuh-python/mag2022/main/CL/term02/04-ClassificationTrees/winequalityN.csv

In [ ]:
data = pd.read_csv('winequalityN.csv')
data.head()

In [ ]:
data.info()

В датасете есть пропуски: дропнем их, иначе торчу будет плохо. 

In [ ]:
data.dropna(inplace=True)

Для простоты сейчас оставим только числовые данные. 

In [ ]:
data.drop('type', axis=1, inplace=True)

Допустим, мы хотим обучить простенькую нейронку на этих данных. Посмотрим распределение классов и их количество:

In [ ]:
data.quality.value_counts()

Давайте укрупним классы: сольем 3, 4 с 5 и 8, 9 с 7 (это, конечно, не дело, но нам пока побаловаться сойдет). 

In [ ]:
data.loc[data['quality'] == 8, 'quality'] = 7
data.loc[data['quality'] == 9, 'quality'] = 7
data.loc[data['quality'] == 3, 'quality'] = 5
data.loc[data['quality'] == 4, 'quality'] = 5

Отделим мухи от котлет, нормализуем данные и для красоты перекодируем классы в 0, 1, 2 (хотя в целом пофиг вроде бы). 

In [ ]:
X = data.drop('quality', axis=1)
y = data.quality

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y

Для таких простых табличных данных можно использовать стандартные Dataset & DataLoader, но мы посмотрим, как можно написать собственный класс Dataset. 

Поделим на трейн и тест обычным sklearn'ом. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

В классе для датасета необходимо перегрузить два метода (помимо init): чтобы экземпляр возвращал свою длину и выдавал пару фичи - ytrue. 

In [ ]:
class MyDataset(Dataset):

    def __init__(self, x, y):
        self.x = torch.tensor(x.values, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
 
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

Проверим, как это будет выглядеть:

In [ ]:
test = DataLoader(dataset=MyDataset(X_train, y_train), batch_size=5, shuffle=True)

In [ ]:
next(iter(test))

Теперь можно собрать трейн и тест. На трейне хотим шаффлить, чтобы было как можно больше вариаций в батчах, а на тесте скорее нет - для детерминированности результата. 

In [ ]:
train_loader = DataLoader(dataset=MyDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=MyDataset(X_test, y_test), batch_size=BATCH_SIZE, shuffle=False)

Зададим параметры и напишем класс с моделью. 

In [ ]:
INPUT_SIZE = 11
HIDDEN_SIZE = 35
OUTPUT_SIZE = 3
LEARNING_RATE = 1e-3
EPOCHS = 100
BATCH_SIZE = 1024 # у нас очень маленький датасет с маленьким набором фич, можем хоть весь целиком в батч запихнуть

In [ ]:
class TorchModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=INPUT_SIZE, out_features=HIDDEN_SIZE)
        self.fc2 = nn.Linear(in_features=HIDDEN_SIZE, out_features=HIDDEN_SIZE)
        self.out = nn.Linear(in_features=HIDDEN_SIZE, out_features=OUTPUT_SIZE)

    def forward(self, x):
        x = nn.LeakyReLU()(self.fc1(x)) # побалуемся с функциями активации
        x = nn.LeakyReLU()(self.fc2(x))
        x = self.out(x)
        return x

Соберем нужные штуки и инициализируем модель

In [ ]:
criterion = nn.CrossEntropyLoss()
accuracy = Accuracy(task='multiclass', num_classes=3)

set_random_seed(42)
model = TorchModel()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

Скопипастим из прошлых тетрадок трейнлуп...

In [ ]:
train_loss_values = []
train_accuracy_values = []
valid_loss_values = []
valid_accuracy = []

def run_train():
    step = 0
    for epoch in range(EPOCHS):
        running_loss = []
        running_acc = []
        for features, label in train_loader:
            # Reset gradients

            output = model(features)
            # Calculate error and backpropagate
            loss = criterion(output, label)
            loss.backward()
            acc = accuracy(output, label).item()

            # Update weights with gradients
            optimizer.step()
            optimizer.zero_grad()

            running_loss.append(loss.item())
            running_acc.append(acc)

        train_loss_values.append(np.mean(running_loss))
        train_accuracy_values.append(np.mean(running_acc))
        if epoch % 20 == 0:
            print(f'EPOCH {epoch}: train_loss: {train_loss_values[-1]}')# train_accuracy_values[-1]))


        # Run validation
        running_loss = []
        running_acc = []
        for features, label in test_loader:
            output = model(features)
            # Calculate error and backpropagate
            loss = criterion(output, label)
            acc = accuracy(output, label).item()

            running_loss.append(loss.item())
            running_acc.append(acc)

        valid_loss_values.append(np.mean(running_loss))
        valid_accuracy.append(np.mean(running_acc))
        if epoch % 20 == 0:
            print(f'EPOCH {epoch}: valid_loss: {valid_loss_values[-1]}, valid_accuracy: {valid_accuracy[-1]}')
        
    return train_loss_values, train_accuracy_values, valid_loss_values, valid_accuracy

In [ ]:
train_loss_values, train_accuracy_values, valid_loss_values, valid_accuracy = run_train()